In [22]:
import os
import numpy as np
from osgeo import gdal, osr
from PIL import Image

def split_merge_tifs(folder_path, ir_file_name, rgb_file_name, red_edge_file_name, block_size):
    ir_tif_file = gdal.Open(os.path.join(folder_path, ir_file_name))
    rgb_tif_file = gdal.Open(os.path.join(folder_path, rgb_file_name))
    red_edge_tif_file = gdal.Open(os.path.join(folder_path, red_edge_file_name))

    block_size = 5

    # Get the geotransform information
    ir_geotransform = ir_tif_file.GetGeoTransform()
    rgb_geotransform = rgb_tif_file.GetGeoTransform()
    red_edge_geotransform = red_edge_tif_file.GetGeoTransform()

    test_1 = 0
    test_2 = 0

    # Iterate over the bounding boxes of each block
    for y in np.arange(ir_geotransform[3], ir_geotransform[3] - ir_tif_file.RasterYSize * ir_geotransform[1], -block_size):
        for x in np.arange(ir_geotransform[0], ir_geotransform[0] + ir_tif_file.RasterXSize * ir_geotransform[1], block_size):
            # Calculate the coordinates of the block
            x_min_block, y_max_block = x, y
            x_max_block, y_min_block = x + block_size, y - block_size
            block_coord = [x + block_size/2, y - block_size/2]

            # Read the pixel values for the block from all three images
            ir_pixel_values = ir_tif_file.ReadAsArray(int((x - ir_geotransform[0]) / ir_geotransform[1]),
                                                        int((ir_geotransform[3] - y) / ir_geotransform[1]),
                                                        int(block_size / ir_geotransform[1]),
                                                        int(block_size / ir_geotransform[1]),
                                                        buf_type=gdal.GDT_Float32)
            rgb_pixel_values = rgb_tif_file.ReadAsArray(int((x - rgb_geotransform[0]) / rgb_geotransform[1]),
                                                        int((rgb_geotransform[3] - y) / rgb_geotransform[1]),
                                                        int(block_size / rgb_geotransform[1]),
                                                        int(block_size / rgb_geotransform[1]))
            red_edge_pixel_values = red_edge_tif_file.ReadAsArray(int((x - red_edge_geotransform[0]) / red_edge_geotransform[1]),
                                                        int((red_edge_geotransform[3] - y) / red_edge_geotransform[1]),
                                                        int(block_size / red_edge_geotransform[1]),
                                                        int(block_size / red_edge_geotransform[1]),
                                                        buf_type=gdal.GDT_Float32)

            # print(ir_pixel_values[1])
            # print(rgb_pixel_values.shape)
            # print(red_edge_pixel_values.shape)

            # Check if any of the pixel values are None
            if ir_pixel_values is None or rgb_pixel_values is None or red_edge_pixel_values is None:
                # Skip processing this block if any pixel values are not available
                continue

            if np.max(rgb_pixel_values) == 0:
                continue

            ir_pixel_values = ir_pixel_values[:-1, :, :]
            rgb_pixel_values = rgb_pixel_values[:-1, :, :]
            red_edge_pixel_values = red_edge_pixel_values[:-1, :, :]

            merged_values = np.concatenate((ir_pixel_values, rgb_pixel_values, red_edge_pixel_values), axis=0)
            # Determine the number of bands
            bands_count = ir_pixel_values.shape[0] + rgb_pixel_values.shape[0] + red_edge_pixel_values.shape[0]

            # Create a new dataset for the 1m x 1m area
            driver = gdal.GetDriverByName('GTiff')

            if driver is None:
                    print("Error: Failed to get GDAL driver.")
                    exit(1)        
            
            res = ir_geotransform[1]

            out_tif = driver.Create(
                os.path.join(folder_path, f"{block_size}m_{block_size}m", f"{int(block_coord[0])}_{int(block_coord[1])}.tif"),
                int(block_size/res), int(block_size/res), bands_count, gdal.GDT_Float32
                
            )

            if out_tif is None:
                # print("Error: Failed to create output TIFF dataset.")
                # exit(1)
                print("Error: Failed to create output TIFF dataset.", test_2)
                test_2 = test_2 + 1
                continue  # Skip processing this block and continue to the next one

            # Set the geotransform
            out_tif.SetGeoTransform((x_min_block, res, 0, y_max_block, 0, -res))
            srs = osr.SpatialReference()            # establish encoding
            srs.ImportFromEPSG(3826)                # TWD97 lat/long
            out_tif.SetProjection(srs.ExportToWkt()) # export coords to file

            # Write the pixel values to the new dataset
            for band_num in range(bands_count):
                band_values = merged_values[band_num]
                out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)
            
            # write to disk
            out_tif.FlushCache()

            # Close the new dataset
            out_tif = None

    # Close the original TIFF file
    tif_file = None
        


In [23]:
 
 for i in range(47,59):
    folder_path = f"d:\\Yehmh\\ZF\\202310\\{i}"
    rgb_file = f"202310_{i}_transparent_mosaic_group1.tif"
    nir_file = f"202310_{i}_transparent_mosaic_nir.tif"
    red_edge_file = f"202310_{i}_transparent_mosaic_red edge.tif"

    # os.makedirs(os.path.join(folder_path, "5m_5m"))
    split_merge_tifs(folder_path, nir_file, rgb_file, red_edge_file, 5)